# 비비빅 학습동아리 11회차_스터디
 - 2024.9.24.(수) 18:10~20:30
 - 국감대비 엑셀 to 파이썬

In [394]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

- 엑셀파일을 먼저 열어서 살펴봅니다.
    - 총 4개 시트, 데이터 갯수 등등.
    - 만약에 이 데이터들이 100만개가 넘는다면?;;;; vlookup 할수있을까?
    - 단순 pivot이 아닌, 중복제거, 데이터 집계 등 작업이 필요한 경우? 어떻게 할건가?

In [395]:
# 엑셀 연계를 위해서는 openpyxl 라이브러리가 필요함
# 아나콘다 설치 시 함께 깔림
#!pip install openpyxl 

In [396]:
# 엑셀 파일 열기 (리소스 할당)
path = './data/sample_data.xlsx'
xls = pd.ExcelFile(path)

# 파일 닫기 (리소스 할당 해제)
# xls_file.close()

In [397]:
# sheet_names를 통해 시트명을 불러올 수 있음
sheets = xls.sheet_names
sheets  # 시트 리스트로 저장

['People', 'Hospitals', 'Diseases', 'Visits']

In [ ]:
# parse를 통해 df로 불러올 수 있음
# 기본형 - 시트명을 지정
df = xls.parse('People')
df.head(5)
# 시트 리스트에서 불러오기
# df = xls.parse(sheets[1])

In [399]:
xls.close()

In [400]:
xls

In [401]:
# 엑셀 파일 열기 (리소스 할당)
path = './data/sample_data_hard.xlsx'
xls = pd.ExcelFile(path)

sheets = xls.sheet_names
sheets  # 총 4개의 시트가 있음을 확인  ['People', 'Hospitals', 'Diseases', 'Visits']

['People', 'Hospitals', 'Diseases', 'Visits']

In [ ]:
# csv처럼 저장되지 않은 엑셀 데이터 불러오기
df = xls.parse('People')
# 엑셀에 제목이 들어있는 등 데이터의 문제가 많음
df.head(5)

In [ ]:
#skiprows 사용. 맨 위 k개 행을 제외
df1 = xls.parse('People', skiprows=2)
df1

In [ ]:
#usecols 사용하여, 제목셀을 무시
df2 = xls.parse(sheets[1], skiprows=3, usecols='A:J')
df2

In [ ]:
# 상병코드 불러오기
# 더 복잡한 형태
df3 = xls.parse(sheets[2], skiprows=5)
df3
# 연번은 index로 처리
# Unnamed: 3	Unnamed: 4	Unnamed: 5	Unnamed: 6 불러오지 않기
# 하단 오류 데이터는 제거

In [ ]:
#최초 nan 값 찾기
df3.연번.isna().idxmax()  ## 30


In [ ]:
# index_col, usecols, skipfooter 사용  ## 43-29 = 13
df3 = xls.parse(sheets[2], 
                skiprows=5, 
                #index_col='연번', 
                index_col=[0],
                usecols='B,C',   # usecols='A,C' , 'A:D', 'A, C:D, G', 'A,B,C'
                skipfooter=14)
df3


In [ ]:
# 진료내역은
# skiprows 사용
df4 = xls.parse(sheets[3], skiprows=2)
df4


# CASE study

## 1. vlookup
진료내역에 수진자 이름, 생년월일, 성별, 주소를 붙여보자

In [409]:
# 빈 데이터프레임
df_exm = pd.DataFrame()
df_exm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [ ]:
# df4.merge(df1)
# 주민번호가 같기 때문에 자동으로 붙는다.
# 명시적으로 붙여주는 방법 (이걸 추천)
df_exm = df4.merge(df1, how='left', left_on='주민번호', right_on='주민번호')
df_exm

In [ ]:
df_exm = df_exm.merge(df2, how='left', left_on='요양기관기호', right_on = '요양기관기호'  )
df_exm
# 중복실행 문제를 방지하려면, suffix 옵션을 사용한다.
# 오늘 다루지는 않겠음 -> gpt문의하면 잘 알려줘요
# 이런 에러가 생긴 경우, df_exm을 처음부터 재구축해주자. df_exm = pd.DataFrame()

In [ ]:
# df3에서 병명코드를 index로 부여했으므로, right_on = '병명코드' 대신 right_index = True 옵션을 사용함.
df_exm = df_exm.merge(df3, how='left', left_on='병명코드', right_index = True  )
df_exm
# 중복실행 문제를 방지하려면, suffix 옵션을 사용한다.
# 오늘 다루지는 않겠음 -> gpt문의하면 잘 알려줘요
# 이런 에러가 생긴 경우, df_exm을 처음부터 재구축해주자. df_exm = pd.DataFrame()

## 1-1. 지역구분 컬럼을 하나 만들어줍시다.

In [ ]:
# 시도/군구 를 한꺼번에 뽑으려면?
#df_exm[['수진자_시도','수진자_구']] = df_exm.주소_x.str.split(' ', n=2, expand=True)[0,1]
df_exm[['수진자_시도','수진자_구']] = df_exm.주소_x.str.split(' ', n=2, expand=True)[[0, 1]]
df_exm[['병원_시도','병원_구']] = df_exm.주소_y.str.split(' ', n=2, expand=True)[[0, 1]]
df_exm

## 2. 데이터에 중복이 있는 지 확인하는 방법  
중복이 있는 지 확인하는것은 매우 중요함 (데이터 검증 영역)

In [ ]:
df1[df1.duplicated()]
# 중복이 없습니다.

In [415]:
# 여러번 쓴다면 함수로 만들어보자
def check_dup(df):
    return df[df.duplicated()]


In [ ]:
check_dup(df4)

In [ ]:
# 특정 컬럼을 기준으로 체크하는 함수
# keep옵션은 False로 해야, 전체 중복된 내용을 체크할 수 있음
# 중복은 2개 행 중복, keep=False가 없는 경우, 둘 중 하나만 반환함.
dup = df1[df1.duplicated(subset=['이름'], keep=False)]
dup

In [ ]:
dup.이름.nunique(), dup.이름.unique(), dup.이름.value_counts()

In [ ]:
df1[df1.이름 == '김지원']

In [420]:
#함수로 만들어볼까요
def check_dup_bycol(df, list):

    print(f'중복검사결과입니다.')

    return df[df.duplicated(subset=list, keep=False)].sort_values(by=list)

In [ ]:
check_dup_bycol(df2,['요양기관기호'])

## 3. groupby를 통한 데이터 집계

In [ ]:
# 수진자 당 진료횟수, 금액, 평균진료금액, 기간은 2021년만
# groupby('컬럼명').agg(딕셔너리...{'컬럼명':'집계값','컬럼명':'집계값'})
df_exm.columns
df = df_exm[['주민번호', '이름','진료일자', '진료비']]
df.groupby('주민번호').agg({
    '이름':'first',
    '진료일자':['count','nunique'],
    '진료비':['sum','max','mean']
})
# groupby를 통해 반환된 결과는 또 다른 df이므로, 이를 이용하여 조건에 따른 필터링  가능함.

### pandas의 group by
| 기능            | 함수명     |      | 기능            | 함수명     |
|-----------------|------------|------|-----------------|------------|
| 평균            | `mean`     |      | 표준편차        | `std`      |
| 합계            | `sum`      |      | 분산            | `var`      |
| 최소값          | `min`      |      | 중앙값          | `median`   |
| 최대값          | `max`      |      | 첫 번째 값      | `first`    |
| 갯수            | `count`    |      | 마지막 값       | `last`     |
| 고유한 값의 수  | `nunique`  |      | 최빈값          | `mode`     |
| 곱              | `prod`     |      | 평균 절대 편차  | `mad`      |
| 표준 오차      | `sem`      |      |         |        |
 

In [ ]:
# 수진자 당 진료횟수, 금액, 평균진료금액, 기간은 2021년만
# groupby('컬럼명').agg(딕셔너리...{'컬럼명':'집계값','컬럼명':'집계값'})
df_exm.columns
df = df_exm[['주민번호', '이름','진료일자', '진료비', '수진자_시도']]
df_group = df.groupby(['주민번호','이름', '수진자_시도']).agg({
    # '이름':'first',
    # '병원이름':'first',
    '진료일자':['count','nunique'],
    '진료비':['sum','max','mean']
    
})
df_group
# groupby를 통해 반환된 결과는 또 다른 df이므로, 이를 이용하여 조건에 따른 필터링  가능함.

## 4. 수진자 지역별, 월별 진료비 현황
# 날짜기준 21.1분기~22.1분기
| 지역명   | 2021/Q1     |              | 2021/Q2     |              | 2021/Q3     |              |
|----------|-------------|--------------|-------------|--------------|-------------|--------------|
|          | 진료비      | 인원         | 진료비      | 인원         | 진료비      | 인원         |
|----------|-------------|--------------|-------------|--------------|-------------|--------------|
| 서울특별시 |             |              |             |              |             |              |
| 부산광역시 |             |              |             |              |             |              |
| 대구광역시 |             |              |             |              |             |              |
| 인천광역시 |             |              |             |              |             |              |
| 광주광역시 |             |              |             |              |             |              |


In [ ]:
# 월별 구분을 위해 진료일자를 날짜 데이터 형식으로 변환
df_exm['진료일자'] = pd.to_datetime(df_exm['진료일자'])
df_exm.info()  # 0   진료일자     3000 non-null   datetime64[ns]    object에서 datetime으로 변환 확인


| 기능          | 메서드/속성                           | 설명                                |
|---------------|---------------------------------------|-------------------------------------|
| 연도 추출     | `df['날짜'].dt.year`                  | 날짜에서 연도만 추출                |
| 월 추출       | `df['날짜'].dt.month`                 | 날짜에서 월만 추출                  |
| 일 추출       | `df['날짜'].dt.day`                   | 날짜에서 일만 추출                  |
| 요일 추출     | `df['날짜'].dt.dayofweek`             | 날짜에서 요일 추출 (0: 월요일)      |
| 요일 이름 추출| `df['날짜'].dt.day_name()`            | 요일의 이름 (예: 'Monday')          |
| 연도-월 추출  | `df['날짜'].dt.to_period('M')`        | 연도-월 추출 (YYYY-MM 형식)         |
| 연도-주 추출  | `df['날짜'].dt.to_period('W')`        | 연도-주 추출                        |
| 연도-분기 추출  | `df['날짜'].dt.to_period('Q')`        | 연도-분기 추출                        |
| 분기 추출     | `df['날짜'].dt.quarter`               | 날짜에서 분기 추출                  |
| 시간 추출     | `df['날짜'].dt.hour`                  | 날짜에서 시간 추출                  |
| 분 추출       | `df['날짜'].dt.minute`                | 날짜에서 분 추출                    |
| 날짜 차이 계산| `df['날짜'] - pd.to_datetime(기준일)` | 날짜 간의 차이 계산                 |
| 원하는 형식 변환 | `df['날짜'].dt.strftime('%Y/%m/%d')` | 날짜를 특정 형식으로 변환            |
| 월의 첫날     | `df['날짜'].dt.to_period('M').start_time` | 해당 월의 첫날 추출             |
| 월의 마지막날 | `df['날짜'].dt.to_period('M').end_time`   | 해당 월의 마지막날 추출          |


In [425]:
# 연도/분기 구분 컬럼을 넣어줍니다. 
df_exm['연도분기'] = df_exm['진료일자'].dt.to_period('Q')

In [426]:
df_exm.columns

Index(['진료일자', '주민번호', '요양기관기호', '병명코드', '진료비', '이름', '생년월일', '성별', '주소_x',
       '병원이름', '의사', '면허번호', '주소_y', '의사 수', '간호사 수', '행정 직원 수', '물리치료사 수',
       '방사선사 수', '병명', '수진자_시도', '수진자_구', '병원_시도', '병원_구', '연도분기'],
      dtype='object')

In [ ]:
df_group = df_exm[['연도분기','주민번호','진료비','수진자_시도', '진료일자']]

# 필터링
# 날짜
con1 = '2021-01-01'
con2 = '2022-03-31'
#((df_group['진료일자'] >= con1) & (df_group['진료일자'] <= con2)).sum() # 1345행
df_group = df_group[(df_group['진료일자'] >= con1) & (df_group['진료일자'] <= con2)]
df_group = df_group.groupby(['연도분기','수진자_시도']).agg({
    '주민번호':'count',
    '진료비':'sum'
})

df_group = df_group.reset_index()
df_group = df_group.pivot_table(columns='연도분기', index='수진자_시도',values=['주민번호','진료비'])

# MultiIndex 레벨 교환 (레벨 0과 레벨 1을 교환)
df_group.columns = df_group.columns.swaplevel(0, 1)

# 첫 번째 레벨(연도/분기)을 기준으로 컬럼 정렬
df_group = df_group.sort_index(axis=1, level=0)
# 두 번째 레벨(진료비, 주민번호)의 순서 조정 (진료비가 먼저 오도록)
df_group = df_group.reindex(columns=['진료비', '주민번호'], level=1)
# 컬럼명 변경
df_group.rename(columns={'주민번호': '진료인원'}, inplace=True)
df_group

## 4. 상위 k, 상위 k% 추출

2023년간 직원 수 상위 병원 k개, 수입 상위 k% 병원 추출

In [ ]:
#  날짜 데이터로 변경
df_exm['진료일자'] = pd.to_datetime(df_exm['진료일자'])
df_exm.info()

In [ ]:
df_exm['직원수'] = df_exm[['의사 수', '간호사 수', '행정 직원 수', '물리치료사 수','방사선사 수']].sum(axis=1)
df_exm.info()

In [ ]:
## 자료발췌
#기간
y = 2023
df_exm[df_exm['진료일자'].dt.year == y]  # 591행

# 병원기준
df_group = df_exm[['연도분기', '요양기관기호','병원이름','주민번호','병명코드','병원_시도','진료비','직원수']]
df_group = df_group.groupby(['요양기관기호','병원이름']).agg({
    '직원수':'sum',
    '진료비':'sum',
    '병원_시도':'first'    
})
df_group

In [ ]:
df_group.sort_values(by='직원수', ascending=False)
df_group.head(10)

In [432]:
# 상위 5% 병원 수 계산
k = int(len(df) * 0.05)
#이하 동일